# The Kinetics of a Diffusion-Controlled Reaction
The title of the notebook should be reflected in the file name. Namely, the file name should be:
*author's initials_title_date.ipynb*
For example:
*DW_fluorescence_analysis_20220202.ipynb*


## Objective
State the purpose of the experiment.


# Setup

## Library import
We import all the required Python libraries

In [ ]:
# File handling
from pathlib import Path
from shutil import unpack_archive
import re

# Data manipulation
import numpy as np
import scipy as sp
import pandas as pd

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("ticks")
sns.despine()

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

# Data import
We retrieve all the required data for the analysis.

In [ ]:
# Unzip your fluorescence data into a subfolder
filename = Path("name of your zip file" + ".zip") # fill in the name of the file without the ZIP extension
dest_folder = Path("name of destination folder") # fill in a name for a subfolder in which to place the text files
unpack_archive(filename, dest_folder)

In [ ]:
# Import all of the files in your new folder. 
# If your files end in something other than ".TXT", make that change here
all_files = list(dest_folder.glob("*.TXT"))

li = [] # Create an empty list to hold the contents of our assorted files
# Import each file in order
# Skip first 31 rows, as data doesn't start until line 32
# Set first (0-th) column (nm) as index
# Default file from fluorometer uses tabs (\t) for separators
for file in all_files:
    df = pd.read_csv(file, skiprows=31, index_col=0, sep='\t') 
    li.append(df)

raw_spectra = pd.concat(li, axis=1) # Use `axis=1` to turn rows of list into columns of dataframe
for file in all_files:
    print(file) # List filenames so we can figure out our column naming in the next step. 
    
raw_spectra.head() # Display the top five rows of our dataframe

In [ ]:
# Here, we use the `re` (Regular Expressions) library to search for patterns in text
# Specifically, we're going to look for the percentage numbers in our filenames. This assumes
# your filenames have a two or three digit number representing the AN percent in your samples.
cols = []
for file in all_files:
    # The next line looks for two or 3 digits in a row, assuming you've placed 
    # the concentration in your file names.
    # You may need to change this pattern, ask for help
    _val = re.findall(r'\d{1,3}',file.stem)[0]
    _val = int(_val)
    
    # This line turns the AN percent into a CBr4 percent. 
    # Modify it if you recorded the CBr4 value.
    cols.append(100-_val) 
    
cols # This is just to check for reasonable values. Order isn't important.

In [ ]:
# Assign the column labels to your dataframe columns, plot the spectra. 
# Make sure your spectra decrease in intensity from 0%–100% CBr4
raw_spectra.columns = cols
raw_spectra.plot()

# Data processing
Put here the core of the notebook. Feel free to further split this section into subsections.

In [ ]:
# Compile a sorted list of maximum intensities from each of our spectra
# The first line grabs the largest value from each spectrum, 
# then divides the largest value in that list by each maximum (calculating I_0/I)
intensities = raw_spectra.max().max()/raw_spectra.max() 

# Now we sort the list of values and convert it to a dataframe for display and plotting
intensities = intensities.sort_index().to_frame()
intensities.columns = ['Intensity'] # Add a proper label to the column
intensities

In [ ]:
# This is an example of a simple function with a "docstring". 
# When writing code, docstrings provide a simple way to document
# the purpose of a function within your code. When a user calls 
# `help(func)` on your function, the docstring will pull up and 
# tell the user how to proceed. 
def p2f(x): 
    '''Convert percentage to floating point numbers'''
    return float(x)/100

init_conc =  # Fill in the concentration (in molarity) of the original CBr4 solution

# The next line uses "list comprehension" to apply a function to each item in a list.
# We convert the index object to a list, then convert each string to a fractional value with p2f, 
# then multiply each fraction by the original concentration
intensities['Concentration (M)'] = [(p2f(item) * init_conc) for item in intensities.index.to_list()]

plt.plot(intensities['Concentration (M)'], intensities['Intensity'], '.')
plt.xlabel('Concentration (M)')
plt.ylabel('$I_0/I$') 
plt.title('[Q] vs. $I_0/I$') # wrapping the input in `$` turns it into LaTeX math
plt.show()

### Collision radius via SES equation

In [ ]:
# Variable setup
# You'll need to create variables for the assorted values you'll use in 
# your calculations (e.g., tau_0, D, etc.)


In [ ]:
## SES: 
#    k = (8 * R_gas * T) / (3 * \eta)
#    k =  4 * \pi * N_A * radius * D / 1000
k_ses = 

# Can format numbers with the `:#.#g` syntax. `g` is an auto-formatter for 
# numbers that automatically trims and converts to scientific form if necessary. 
print(f"The reaction constant with the SES equation is {k_ses:g} 1/(M*s).") 

# Now calculate the radius
radius_ses = 
print(f"The reaction radius is {radius_ses:g} dm")

### Collision radius via SES equation

In [ ]:
# Variable setup
# Fill in appropriate variables for the SES equation. 

In [ ]:
## SES: k = (8 * R * T) / (3 * \eta)
k_ses = 
print(f"The reaction constant with the SES equation is {k_ses:g} 1/(M*s).") 

radius_ses = 
print(f"The reaction radius is {radius_ses:g} dm")

In [ ]:
# Now we need to plot a line based on the SES calculations

# We start by making a range of concentration values. 
# `linspace` is an ideal function as it makes evenly spaced 
# points between the start and stop values (50, by default). 
conc_data = np.linspace(0.000,0.015)
ses_fit = tau_0 * k_ses * xdata + 1

plt.plot(xdata,ses_fit,'-',
        intensities["Concentration (M)"][:7], intensities["Intensity"][:7], ".")
plt.xlabel('Concentration (M)')
plt.ylabel('$I_0/I$')
plt.title('$I_0/I$ vs. [Q]')
plt.show()

### Collision radius via Stern-Vollmer relation (no transient term)

In [ ]:
# We're supposed to look at a linear fit for just the first few points. 
# Seaborn's `lmfit` is a good tool for this. By changing the number of 
# points used from intensities, you can see how the fit degrades as you 
# use more points. Change the value of `4` to smaller and larger numbers 
# and see how the fit changes
sns.lmplot(data=intensities[:4],x="Concentration (M)", y="Intensity")

In [ ]:
# Again, we need to make a linear fit of the data, so we'll 
# use the polyfit function from numpy. 
fit_coeff, fit_cov = np.polyfit(intensities["Concentration (M)"][:4], intensities["Intensity"][:4], 1, cov=True)

fit_err = np.sqrt(np.diag(fit_cov))

print(f"slope: {fit_coeff[0]/tau_0:1.4g}, std. err.: {fit_err[0]/tau_0:g}\n\
intercept: {fit_coeff[1]:g}, std. err.: {fit_err[1]:g}")
radius_fit = fit_coeff[0]/tau_0 * 1000 / (4 * sp.constants.Avogadro  * np.pi * diffusion_hexane )
print(f"The reaction radius is {radius_fit:g} dm")

In [ ]:
# We'll compare the fit from the previous method to one using the curve_fit 
# routine from scipy.optimize. In your summary, comment on the differences 
# (and why they exist). The key is in the y-intercept…

def sv_func(conc, k_q):
    return k_q * tau_0 * conc + 1


from scipy.optimize import curve_fit

# Need to provide initial guess for parameter…
sv_coeff, sv_cov = curve_fit(sv_func, intensities["Concentration (M)"][:4], intensities["Intensity"][:4], p0=2e10)

sv_err = np.sqrt(np.diag(sv_cov))

print(sv_coeff, "\n", sv_err)
radius_sv = sv_coeff[0] * 1000 / (4 * sp.constants.Avogadro  * np.pi * diffusion_hexane )
print(f"The reaction radius is {radius_sv:g} dm")

### Collision radius via full Stern-Vollmer relation (with transient term)

In [ ]:
def a_const(R, conc):
    return # fill in the equation for the "a" constant

def b_const(R, conc): 
    return # fill in the equation for the "b" constant

def y_const(R, conc):
    return # fill in the equation for the "Y" constant
            # you must input `a` and `b` as 
            # `a_const(R, conc)` and `b_const(R, cons)`

In [ ]:
# It's worth playing with values of R and conc to see how they affect Y
# Experiment with them here. 
print(y_const(5e-8, 1.5e-2))

In [ ]:
def intensity(conc, R):
    return # fill in the equation for the intensity (I_0/I) as a function
           # of conc, R, and y_const(conc, R)

sv_coeff_2 , sv_cov_2 = curve_fit(intensity, 
                                  intensities["Concentration (M)"], 
                                  intensities["Intensity"], 
                                  p0= []# Enter an initial guess for the parameters (just R, in cm, in this case)
                                 )

sv_err_2 = np.sqrt(np.diag(sv_cov_2))

print(f"R = {sv_coeff_2[0]:g} ± {sv_err_2[0]:.2e} cm")

# Create a plot that shows your fit compared to the experimental data


## Results
Describe and comment the most important results.


# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2